# run this in google colab cause idk how to make it run in my charm. If you're having an issue downloading leftImg8bit_trainvaltest_foggy.zip

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install necessary packages
!pip install requests --quiet
!pip install beautifulsoup4 --quiet

In [ ]:
import requests, os, time
from bs4 import BeautifulSoup

# Create directory in Google Drive for the dataset
save_dir = '/content/drive/My Drive/SDSU/CS559/cityscapes_dataset'
os.makedirs(save_dir, exist_ok=True)

# Set Cityscapes login credentials
username = 'username'
password = 'password'

# Create a session to maintain cookies
session = requests.Session()


# get the login page
login_url = 'https://www.cityscapes-dataset.com/login/'
response = session.get(login_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
login_data = {
    'username': username,
    'password': password,
    'submit': 'Login'
}

# Perform the login
response = session.post(login_url, data=login_data)

In [ ]:
# Verify login success
if "Login failed" in response.text or "Please login" in response.text:
    print("Login failed. Check your credentials.")
else:
    print("Login appears successful!")

    # Now access the downloads page
    downloads_url = 'https://www.cityscapes-dataset.com/downloads/'
    response = session.get(downloads_url)

    # Check if we can see the downloads page content
    if "available downloads" in response.text.lower():
        print("Successfully accessed downloads page")

        # Find the download links
        soup = BeautifulSoup(response.text, 'html.parser')

        # Print all links to help identify the correct ones
        print("Available download links:")
        for link in soup.find_all('a'):
            if 'file-handling' in link.get('href', ''):
                print(f"Link: {link.get('href')} - Text: {link.text}")

        # Example: Download leftImg8bit_trainvaltest_foggy.zip
        # Use the correct URL from the printed links
        download_url = 'https://www.cityscapes-dataset.com/file-handling/?packageID=29'  # Replace X with correct ID

        # Extract just the filename without the URL
        filename = 'leftImg8bit_trainvaltest_foggy.zip'  # Set the correct filename

        print(f"\nStarting download of {filename}...")
        start_time = time.time()

        # Set proper headers to appear as a browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': downloads_url
        }

        response = session.get(download_url, headers=headers, stream=True)

        # Check response
        content_type = response.headers.get('Content-Type', '')
        content_length = response.headers.get('Content-Length', '')

        print(f"Content-Type: {content_type}")
        print(f"Content-Length: {content_length} bytes")

        if 'application/zip' in content_type or 'application/octet-stream' in content_type or int(content_length if content_length else 0) > 1000000:
            # Save the file with progress updates
            file_path = os.path.join(save_dir, filename)
            print(f"Saving to: {file_path}")

            total_size = int(content_length) if content_length else 0
            downloaded = 0
            last_print_time = time.time()

            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024*1024):  # 1MB chunks
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)

                        # Only print progress every 5 seconds
                        current_time = time.time()
                        if current_time - last_print_time > 5 or downloaded == total_size:
                            if total_size > 0:
                                percent = downloaded / total_size * 100
                                print(f"Downloaded {downloaded/1024/1024:.1f} MB of {total_size/1024/1024:.1f} MB ({percent:.1f}%)")
                            else:
                                print(f"Downloaded {downloaded/1024/1024:.1f} MB")
                            last_print_time = current_time

            end_time = time.time()
            total_time = end_time - start_time
            print(f"Download completed in {total_time:.1f} seconds")

            # Verify file size
            file_size = os.path.getsize(file_path)
            print(f"Downloaded file size: {file_size/1024/1024:.1f} MB")

            if total_size > 0 and file_size < total_size * 0.9:
                print("WARNING: Downloaded file is significantly smaller than expected size. It may be incomplete.")
        else:
            print("Error: Did not receive a file. The response may be an error page or login redirect.")
            print("First 500 characters of response:")
            print(response.text[:500])
    else:
        print("Could not access downloads page correctly. Authentication may have failed.")
        print("First 500 characters of response:")
        print(response.text[:500])